In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 15831, done.
remote: Total 15831 (delta 0), reused 0 (delta 0), pack-reused 15831
Receiving objects: 100% (15831/15831), 14.48 MiB | 20.15 MiB/s, done.
^C
[Errno 2] No such file or directory: 'yolov5'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
if not os.path.isfile('/content/drive/MyDrive/Kaggle/obj_detection/dataset/data.yaml'): 
  %cp /content/yolov5/data/coco.yaml /content/drive/MyDrive/Kaggle/obj_detection/dataset/data.yaml

In [ ]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders
import os

if not os.path.isdir('/content/drive/MyDrive/Kaggle/obj_detection/dataset_split'):
  splitfolders.ratio('/content/drive/MyDrive/Kaggle/obj_detection/dataset/train/', 
                     output="/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/", 
                     seed=7777,
                     ratio=(0.9,0.1))
  %cp /content/drive/MyDrive/Kaggle/obj_detection/dataset/test /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/test


Copying files: 1204 files [00:47, 25.33 files/s]

cp: -r not specified; omitting directory '/content/drive/MyDrive/Kaggle/obj_detection/dataset/test'


data.yaml 세팅

In [ ]:
TRAIN_DIR = '/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/train/train'
VALID_DIR = '/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/val/val'
TEST_DIR = '/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/test'

In [ ]:
from glob import glob

train_img_list = glob(f'{TRAIN_DIR}/*.jpg')
valid_img_list = glob(f'{VALID_DIR}/*.jpg')
test_img_list = glob(f'{TEST_DIR}/*.jpg')

print(len(train_img_list), len(valid_img_list), len(test_img_list))

0 0 0


In [ ]:
import yaml

with open('/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list)+'\n')

with open('/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/valid.txt', 'w') as f:
    f.write('\n'.join(valid_img_list)+'\n')

with open('/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/test.txt', 'w') as f:
    f.write('\n'.join(test_img_list)+'\n')

In [ ]:
with open("/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/data.yaml", 'r') as f:
  data = yaml.load(f, Loader=yaml.FullLoader)

NameError: ignored

In [ ]:
data['train'] = "/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/images/train"
data['val'] = "/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/images/val"
data['test'] = "/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/images/test"
data['nc'] = 4
data['names']= {0:'Buffalo',1:'Elephant',2:'Rhinoceros',3:'Zebra'}
data['path'] = "/content/drive/MyDrive/Kaggle/obj_detection/dataset_split"

with open("/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/data.yaml", 'w') as f:
  yaml.dump(data, f)

with open("/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/data.yaml") as f:
  temp =  yaml.load(f, Loader=yaml.FullLoader)
  display(temp)

{'download': "from utils.general import download, Path\n\n\n# Download labels\nsegments = False  # segment or box labels\ndir = Path(yaml['path'])  # dataset root dir\nurl = 'https://github.com/ultralytics/yolov5/releases/download/v1.0/'\nurls = [url + ('coco2017labels-segments.zip' if segments else 'coco2017labels.zip')]  # labels\ndownload(urls, dir=dir.parent)\n\n# Download data\nurls = ['http://images.cocodataset.org/zips/train2017.zip',  # 19G, 118k images\n        'http://images.cocodataset.org/zips/val2017.zip',  # 1G, 5k images\n        'http://images.cocodataset.org/zips/test2017.zip']  # 7G, 41k images (optional)\ndownload(urls, dir=dir / 'images', threads=3)\n",
 'names': {0: 'Buffalo', 1: 'Elephant', 2: 'Rhinoceros', 3: 'Zebra'},
 'nc': 4,
 'path': '/content/drive/MyDrive/Kaggle/obj_detection/dataset_split',
 'test': '/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/images/test',
 'train': '/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/images/train',
 '

전처리

- train_output.csv 전처리 필요
- .yaml의 path 경로인 /content/drive/MyDrive/Kaggle/obj_detection/dataset_split
아래 labels 파일에 txt로 labeling 필요

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/train_output.csv')
df.head()

,filename,label
0,1,1 0.438281 0.529551 0.814063 0.704492
1,2,3 0.541016 0.524324 0.849219 0.672973
2,3,2 0.492500 0.517045 0.695000 0.725000
3,4,2 0.585156 0.525000 0.692187 0.785135
4,5,3 0.385547 0.645270 0.614844 0.698649\n3 0.559...


In [ ]:
label = df.loc[:, 'label'].to_list()
label 

['1 0.438281 0.529551 0.814063 0.704492',
 '3 0.541016 0.524324 0.849219 0.672973',
 '2 0.492500 0.517045 0.695000 0.725000',
 '2 0.585156 0.525000 0.692187 0.785135',
 '3 0.385547 0.645270 0.614844 0.698649\n3 0.559375 0.682432 0.445313 0.629730\n3 0.143359 0.382432 0.286719 0.318919',
 '1 0.429688 0.596737 0.590625 0.778555',
 '3 0.516797 0.504054 0.380469 0.929730',
 '0 0.476526 0.520833 0.298122 0.404167',
 '2 0.461184 0.470783 0.435774 0.877711\n2 0.813725 0.296988 0.370948 0.530120',
 '3 0.700781 0.626042 0.532813 0.739583',
 '1 0.698000 0.608696 0.500000 0.561873\n1 0.275000 0.608696 0.374000 0.561873',
 '2 0.680000 0.648750 0.263333 0.532500\n2 0.309167 0.548750 0.415000 0.742500',
 '1 0.643750 0.630653 0.293750 0.552764\n1 0.548438 0.218593 0.281250 0.316583\n1 0.172656 0.326633 0.298438 0.361809',
 '0 0.589630 0.555241 0.773333 0.878187',
 '3 0.159766 0.586486 0.108594 0.197297\n1 0.751953 0.556081 0.319531 0.612162',
 '2 0.465833 0.496250 0.751667 0.784167',
 '3 0.844531 0.7

In [ ]:
# label format
import os

path = os.listdir('/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/train/train')
train_list = sorted([file for file in path])
for i in range(len(train_list)):
  train_list[i] = train_list[i].replace('.jpg','')

path = os.listdir('/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/val/val')
val_list = sorted([file for file in path])
for i in range(len(val_list)):
  val_list[i] = val_list[i].replace('.jpg','')

print ("train_list: {}".format(train_list))
print ("val_list: {}".format(val_list))

train_list: ['0001', '0002', '0003', '0005', '0007', '0008', '0009', '0011', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0026', '0027', '0028', '0029', '0030', '0031', '0032', '0033', '0034', '0035', '0036', '0037', '0038', '0040', '0041', '0043', '0044', '0045', '0046', '0047', '0048', '0049', '0050', '0051', '0053', '0054', '0055', '0057', '0058', '0059', '0060', '0061', '0063', '0064', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0075', '0076', '0077', '0078', '0079', '0080', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0088', '0089', '0090', '0092', '0093', '0094', '0095', '0096', '0097', '0099', '0100', '0102', '0103', '0104', '0105', '0106', '0107', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0117', '0118', '0119', '0120', '0121', '0122', '0123', '0124', '0128', '0129', '0130', '0131', '0132', '0133', '0134', '0135', '0137', '0138', '0139', '0140', '0141', '0142', '0143', '01

In [ ]:
%cd /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/labels/train

/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/labels/train


In [ ]:
for i in range(len(train_list)):
  f = open(f'{train_list[i]}.txt', 'w')
  index = int(train_list[i])-1
  to_text = str(label[index])
  f.write(to_text)
  f.close()

In [ ]:
%cd /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/labels/val

/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/labels/val


In [ ]:
for i in range(len(val_list)):
  f = open(f'{val_list[i]}.txt', 'w')
  index = int(val_list[i])-1
  to_text = str(label[index])
  f.write(to_text)
  f.close()

test set label format

In [ ]:
import pandas as pd

test_frame = pd.read_csv('/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/test_output.csv')
test_frame.head()

,filename,label
0,1205,1 0.662500 0.482394 0.606250 0.607981\n1 0.207...
1,1206,3 0.703906 0.629977 0.367188 0.707260\n3 0.331...
2,1207,2 0.511250 0.540338 0.832500 0.615385
3,1208,0 0.551562 0.545946 0.484375 0.494595
4,1209,3 0.429000 0.564000 0.702000 0.749333\n3 0.402...


In [ ]:
test_label = test_frame.loc[:, 'label'].to_list()
test_label 

['1 0.662500 0.482394 0.606250 0.607981\n1 0.207813 0.696009 0.196875 0.218310\n1 0.450000 0.529343 0.481250 0.457746',
 '3 0.703906 0.629977 0.367188 0.707260\n3 0.331250 0.567916 0.234375 0.737705\n3 0.239063 0.421546 0.231250 0.538642\n3 0.544531 0.585480 0.251563 0.637002\n3 0.563281 0.278689 0.335938 0.210773\n3 0.263281 0.174473 0.260937 0.147541\n3 0.203125 0.063232 0.103125 0.121780\n3 0.500781 0.142857 0.267188 0.173302\n3 0.893750 0.124122 0.115625 0.173302\n3 0.706250 0.154567 0.334375 0.210773\n3 0.060156 0.600703 0.117188 0.545667',
 '2 0.511250 0.540338 0.832500 0.615385',
 '0 0.551562 0.545946 0.484375 0.494595',
 '3 0.429000 0.564000 0.702000 0.749333\n3 0.402000 0.137333 0.364000 0.269333',
 '3 0.357812 0.493750 0.390625 0.454167\n3 0.580469 0.463542 0.379688 0.406250',
 '3 0.639453 0.510135 0.541406 0.714865\n3 0.411719 0.581081 0.656250 0.751351',
 '0 0.398998 0.657428 0.547579 0.490022',
 '2 0.442188 0.506757 0.676563 0.981081',
 '0 0.463500 0.496667 0.665000 0.7960

In [ ]:
test_list = sorted(test_frame.loc[:, 'filename'].to_list())

print ("test_list: {}".format(test_list))

test_list: [1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369

In [ ]:
%cd /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/labels/test

/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/labels/test


In [ ]:
for i in range(len(test_list)):
  f = open(f'{test_list[i]}.txt', 'w')
  to_text = str(test_label[i])
  f.write(to_text)
  f.close()

모델 구성

학습

In [ ]:
!pwd
%cd /content/yolov5

/content/yolov5
/content/yolov5


In [ ]:
 #!python train.py --help

In [ ]:
!python train.py --img 1280 --batch 16 --epochs 100 --patience 15 --data '/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/data.yaml' --weights yolov5s.pt --project '/content/drive/MyDrive/Kaggle/obj_detection/model' --name 'trained' --exist-ok

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/Kaggle/obj_detection/model, name=trained, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=15, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1

학습 재게

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

In [ ]:
!python train.py --resume /content/drive/MyDrive/Kaggle/obj_detection/model/trained/weights/last.pt

train: weights=yolov5s.pt, cfg=, data=data/coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=/content/drive/MyDrive/Kaggle/obj_detection/model/trained/weights/last.pt, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0,

검증


In [ ]:
!python val.py --data "/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/data.yaml" --img 1280 --weights "/content/drive/MyDrive/Kaggle/obj_detection/model/trained/weights/best.pt"

val: data=/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/data.yaml, weights=['/content/drive/MyDrive/Kaggle/obj_detection/model/trained/weights/best.pt'], batch_size=32, imgsz=1280, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/labels/val.cache... 121 images, 0 backgrounds, 0 corrupt: 100% 121/121 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 4/4 [00:10<00:00,  2.73s/it]
                   all        121        205      0.861      0.861      0.914      0.

test set 검증

In [ ]:
!cp -r /content/yolov5/runs/val/exp3 /content/drive/MyDrive/Kaggle/obj_detection/model

In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
!python val.py --task "test" --data /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/data.yaml --weights /content/drive/MyDrive/Kaggle/obj_detection/model/trained/weights/best.pt --img 1280 --save-txt

val: data=/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/data.yaml, weights=['/content/drive/MyDrive/Kaggle/obj_detection/model/trained/weights/best.pt'], batch_size=32, imgsz=1280, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/labels/test... 300 images, 0 backgrounds, 0 corrupt: 100% 300/300 [00:01<00:00, 179.47it/s]
test: New cache created: /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/labels/test.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 10/10 [0

In [ ]:
!cp -r /content/yolov5/runs/val/exp6 /content/drive/MyDrive/Kaggle/obj_detection/model/test

예측

In [ ]:
!python detect.py --weights "/content/drive/MyDrive/Kaggle/obj_detection/model/trained/weights/best.pt" --source "/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/images/test"

detect: weights=['/content/drive/MyDrive/Kaggle/obj_detection/model/trained/weights/best.pt'], source=/content/drive/MyDrive/Kaggle/obj_detection/dataset_split/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
image 1/300 /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/images/test/1205.jpg: 448x640 3 Elephants, 40.4ms
image 2/300 /content/drive/MyDrive/Kaggle/obj_detection/dataset_split/images/test/1206.jpg: 448x640 9 Zebras, 8.8ms
image 3/300 /cont

In [ ]:
!cp -r /content/yolov5/runs/detect/exp2 /content/drive/MyDrive/Kaggle/obj_detection/model/test/detect

In [ ]:
!python detect.py --weights yolov5s.pt --source 0

detect: weights=['yolov5s.pt'], source=0, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

100% 14.1M/14.1M [00:00<00:00, 74.8MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

[ WARN:0@7.807] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@7.807] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range
Traceback (most recent call last):
  File "/content/yolov5/detect.py", lin